<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Digital-Scotch-Sommelier-Capstone" data-toc-modified-id="Digital-Scotch-Sommelier-Capstone-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Digital Scotch Sommelier Capstone</a></span></li><li><span><a href="#Raw-Input-Questions" data-toc-modified-id="Raw-Input-Questions-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Raw Input Questions</a></span><ul class="toc-item"><li><span><a href="#--What-is-your-price-range?" data-toc-modified-id="--What-is-your-price-range?-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>- What is your price range?</a></span></li><li><span><a href="#--What-category-would-you-prefer?" data-toc-modified-id="--What-category-would-you-prefer?-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>- What category would you prefer?</a></span></li><li><span><a href="#--What-ABV-level-would-you-prefer?" data-toc-modified-id="--What-ABV-level-would-you-prefer?-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>- What ABV level would you prefer?</a></span></li><li><span><a href="#--What-tasting-notes-do-you-prefer?" data-toc-modified-id="--What-tasting-notes-do-you-prefer?-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>- What tasting notes do you prefer?</a></span></li></ul></li><li><span><a href="#Preference-Selection" data-toc-modified-id="Preference-Selection-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Preference Selection</a></span></li><li><span><a href="#Recommendations" data-toc-modified-id="Recommendations-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Recommendations</a></span></li></ul></div>

# Digital Scotch Sommelier Capstone

In [46]:
# Imports
from __future__ import print_function

import pandas as pd
import re
import numpy as np
import seaborn as sns
import decimal
import ipywidgets as widgets
import matplotlib.pyplot as plt
import nltk

from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.feature_extraction.text import CountVectorizer

from bs4 import BeautifulSoup

from nltk.tokenize import word_tokenize, regexp_tokenize
from nltk.corpus import stopwords

from ipywidgets import interact, interactive, fixed, interact_manual
from ipywidgets import widgets, interact, interact_manual, fixed
from IPython.display import display, clear_output, HTML

sns.set()
pd.options.mode.chained_assignment = None
pd.set_option('display.max_colwidth', -1)
%matplotlib inline

# Raw Input Questions

---
## - What is your price range? 


| Below 50 | 50 to 100 | 150 to 200 | 200 to 500 | Above 500 |
|:--------:|:---------:|:----------:|:----------:|:---------:|

---

In [47]:
def price_range_f(price_range):
    return price_range

In [48]:
price_range = interactive(
    price_range_f,
    price_range=[
        'Below 50', '50 to 100', '150 to 200', '200 to 500', 'Above 500'
    ])

---
## - What category would you prefer?


| Blended Scotch Whisky | Single Malt Scotch | Blended Malt Scotch Whisky | Grain Scotch Whisky | Single Grain Whisky |
|:-:|:-:|:-:|:-:|:-:|

---

In [49]:
def category_f(category_range):
    return category_range

In [50]:
category_range = interactive(
    category_f,
    category_range=[
        'Blended Scotch Whisky', 'Single Malt Scotch',
        'Blended Malt Scotch Whisky', 'Grain Scotch Whisky',
        'Single Grain Whisky'
    ])


---
## - What ABV level would you prefer?

| Below 40% | 40% | Above 40% |
|:--------:|:---------:|:----------:|

---


In [51]:
def abv_range_f(abv_range):
    return abv_range

In [52]:
abv_range = interactive(
    abv_range_f, abv_range=['Below 40%', '40%', 'Above 40%'])


---
## - What tasting notes do you prefer?

| taste |  taste |  taste |   taste  |  taste  |  taste |
|:------:|:-------:|:------:|:--------:|:------:|:-------:|
| earth |  fruit |  honey |   cream  |  malt  |  herbal |
|  wood |  floral |  nutty |  ginger  | citrus | vanilla |
|  spicy |  peaty  |   dry  |  toffee  | pepper | caramel |
|  malt |  silk  | syrup |   toast  |   oil  |         |
|  wine | cereal |  salty | cinnamon |  sweet |         |

---
---

In [53]:
def taste_notes1(first_taste):
    return first_taste

In [54]:
def taste_notes2(second_taste):
    return second_taste

In [55]:
def taste_notes3(third_taste):
    return third_taste

In [56]:
first_taste = interactive(
    taste_notes1,
    first_taste=[
        'earth', 'wood', 'spicy', 'malt', 'wine', 'fruity', 'floral',
        'peaty', 'silk', 'cereals', 'honey', 'nutty', 'dry', 'syrup',
        'salty', 'cream', 'ginger', 'toffee', 'toast', 'cinnamon', 'malt',
        'citrus', 'pepper', 'oil', 'sweet', 'herbal', 'vanilla', 'caramel'
    ])

In [57]:
second_taste = interactive(
    taste_notes2,
    second_taste=[
        'earth', 'wood', 'spicy', 'malt', 'wine', 'fruity', 'floral',
        'peaty', 'silk', 'cereals', 'honey', 'nutty', 'dry', 'syrup',
        'salty', 'cream', 'ginger', 'toffee', 'toast', 'cinnamon', 'malt',
        'citrus', 'pepper', 'oil', 'sweet', 'herbal', 'vanilla', 'caramel'
    ])

In [58]:
third_taste = interactive(
    taste_notes3,
    third_taste=[
        'earth', 'wood', 'spicy', 'malt', 'wine', 'fruity', 'floral',
        'peaty', 'silk', 'cereals', 'honey', 'nutty', 'dry', 'syrup',
        'salty', 'cream', 'ginger', 'toffee', 'toast', 'cinnamon', 'malt',
        'citrus', 'pepper', 'oil', 'sweet', 'herbal', 'vanilla', 'caramel'
    ])

In [59]:
# Reading in the scotch reviews data received from Kaggle
scotch = pd.read_csv('./dataset/scotch_review.csv')

In [60]:
# scotch.head()

In [61]:
# scotch['price']

In [62]:
# Dropping Unnamed: 0 due to being unneeded
# and currency due to all are in USD
scotch.drop(['Unnamed: 0', 'currency'], axis=1, inplace=True)

In [63]:
# Verify correct columns have dropped
# scotch.head()

In [64]:
# Renaming column for simplicity
scotch.rename(index=str, columns={'review.point': 'points'}, inplace=True)

In [65]:
# Seeing the dtypes of each column
# scotch.info()

In [66]:
# Sorting the price
# sorted(scotch.price)

In [67]:
# Extracting and just the numbers from the price
scotch['price'] = scotch['price'].str.extract('(\d\d\d*|\d)')
# scotch['price']

In [68]:
# Changing all strings to floats
scotch['price'] = scotch['price'].astype(str).astype(float)

In [69]:
# scotch.dtypes

In [70]:
# scotch.shape

In [71]:
# Finding all the scotches that have an abv associated
# and breaking them into a new column

scotch['abv'] = scotch['name'].str.extract('(\d\d%|\d\d.\d%)')

# print('%.1f%% of the scotches have an abv associated with them.' %((1-(scotch['abv'].isnull().sum()/len(scotch['abv'])))*100))

In [72]:
# Since there are so few nan I will just drop them
scotch['abv'].dropna(inplace=True)

In [73]:
# Because abv was with the name column before, it was a string
# Converted it to a float
scotch['abv'] = scotch['abv'].astype(str).str[:-1].astype(float)

In [74]:
# Finding the nan
# scotch['abv'].isnull().sum()

In [75]:
# Dropping the nan
scotch['abv'].dropna(inplace=True)

In [76]:
# Verifying nan drop
# scotch['abv'].isnull().sum()

In [77]:
# Another verification for fun
# print('%.1f%% of the scotches have an abv associated with them.' %((1-(scotch['abv'].isnull().sum()/len(scotch['abv'])))*100))

In [78]:
# scotch.head()

In [79]:
# Separating prices into 6 understandable bins
bins_price = [0, 50, 100, 150, 200, 500, np.inf]
price = [
    'Below 50', '50 to 100', '100 to 150', '150 to 200', '200 to 500',
    'Above 500'
]

scotch['price_categories'] = pd.cut(scotch['price'], bins_price, labels=price)

In [80]:
# Separating abv into 3 understandable bins
bins_abv = [0, 39, 40, np.inf]
abv = ['Below 40%', '40%', 'Above 40%']

scotch['abv_categories'] = pd.cut(scotch['abv'], bins_abv, labels=abv)

In [81]:
# scotch.shape

# Preference Selection

In [100]:
# Select each other categories
display(price_range)
display(category_range)
display(abv_range)
display(first_taste)
display(second_taste)
display(third_taste)

interactive(children=(Dropdown(description='price_range', index=3, options=('Below 50', '50 to 100', '150 to 2…

interactive(children=(Dropdown(description='category_range', index=1, options=('Blended Scotch Whisky', 'Singl…

interactive(children=(Dropdown(description='abv_range', index=2, options=('Below 40%', '40%', 'Above 40%'), va…

interactive(children=(Dropdown(description='first_taste', index=2, options=('earth', 'wood', 'spicy', 'malt', …

interactive(children=(Dropdown(description='second_taste', index=26, options=('earth', 'wood', 'spicy', 'malt'…

interactive(children=(Dropdown(description='third_taste', index=27, options=('earth', 'wood', 'spicy', 'malt',…

In [101]:
price_range_drop = price_range.result
category_range_drop = category_range.result
abv_range_drop = abv_range.result
first_taste_drop1 = first_taste.result
second_taste_drop2 = second_taste.result
third_taste_drop3 = third_taste.result

In [102]:
# # Should change with every change of a drop down
# print(price_range_drop)
# print(category_range_drop)
# print(abv_range_drop)
# print(first_taste_drop1)
# print(second_taste_drop2)
# print(third_taste_drop3)

In [103]:
# These should be the inputs your scotch preferances so they
# can be made into a dataframe
scotch_pref = pd.DataFrame({
    'price_range_drop': [price_range_drop],
    'category_range_drop': [category_range_drop],
    'abv_range_drop': [abv_range_drop],
    'first_taste_drop1': [first_taste_drop1],
    'second_taste_drop2': [second_taste_drop2],
    'third_taste_drop3': [third_taste_drop3]
},
                           index=[0]).T
# scotch_pref

In [104]:
price_pref = scotch.loc[scotch['price_categories'] == price_range_drop]

In [105]:
category_pref = scotch.loc[scotch['category'] == category_range_drop]

In [106]:
abv_pref = scotch.loc[scotch['abv_categories'] == abv_range_drop]

In [107]:
# price_pref.head()

In [108]:
# category_pref.head()

In [109]:
# abv_pref.head()

In [110]:
abv_price_pref = price_pref.merge(abv_pref, how='inner', on=['name'])

In [111]:
abv_price_cat_pref = abv_price_pref.merge(
    category_pref, how='inner', on=['name'])

In [112]:
# abv_price_cat_pref.head()

In [113]:
basic_pref = abv_price_cat_pref[[
    'name', 'category', 'points', 'price', 'abv', 'description'
]]

In [114]:
# basic_pref.shape

In [115]:
# len(basic_pref)

# Recommendations

In [116]:
recommendation = []
for _ in range(0, len(basic_pref)):
    result = {}
    if first_taste_drop1 in basic_pref['description'][
            _] and second_taste_drop2 in basic_pref['description'][
                _] and third_taste_drop3 in basic_pref['description'][_]:
        result['final_rec'] = basic_pref.iloc[[_]]

        if first_taste_drop1 in basic_pref['description'][
                _] and second_taste_drop2 in basic_pref['description'][_]:
            result['final_rec'] = basic_pref.iloc[[_]]

            if first_taste_drop1 in basic_pref['description'][
                    _] and third_taste_drop3 in basic_pref['description'][_]:
                result['final_rec'] = basic_pref.iloc[[_]]

                if second_taste_drop2 in basic_pref['description'][
                        _] and third_taste_drop3 in basic_pref['description'][
                            _]:
                    result['final_rec'] = basic_pref.iloc[[_]]

                    if first_taste_drop1 in basic_pref['description'][
                            _] or second_taste_drop2 in basic_pref[
                                'description'][
                                    _] or third_taste_drop3 in basic_pref[
                                        'description'][_]:
                        result['final_rec'] = basic_pref.iloc[[_]]

                        recommendation.append(result['final_rec'])
recommendation = pd.concat(recommendation)
recommendation.head()

,name,category,points,price,abv,description
1,"Brora, 30 year old (2009 Release), 53.2%",Single Malt Scotch,95,400.0,53.2,"This whisky has all the positive aspects of a very mature whisky (depth, complexity) without all the negative ones (excessive oak, one-dimensional). Very clean, but oily in texture, with honeyed vanilla, caramel, citrus (tangerine, orange, lemon), nectarine, olive brine, black pepper, ginger, cut grass, mustard seed, and just the hint of teasing smoke. Briny, spicy finish. Wonderful!"
68,"The Dalmore Distillery Exclusive 1991, 59%",Single Malt Scotch,91,236.0,59.0,"Staff members at Dalmore selected this distillery-exclusive bottling, which has been drawn from American white oak cask number 446, and bottled at cask strength. The out-turn comprises 450 bottles. Refined, polished oak on the nose. Honey, marzipan, caramel, and vanilla. Fragrant, with old leather and over-ripe oranges when water is added. Warm leather, apricots, orange marmalade, cocoa powder, and developing spice on the palate. A long, spicy, citric finish. Finally, licorice. £150"


In [73]:
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script><a href="javascript:code_toggle()">Hide</a> code.''')